In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import sys
import unicodedata
import line_profiler
%load_ext line_profiler

In [4]:
api_list = []

#12346120
for i in range(12346120, 12346126):
    url = "http://cogcc.state.co.us/cogis/FacilityDetail.asp?facid=" + str(i) + "&type=WELL"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    fields = soup.find_all('font')
    spud_d = {}
    
    if fields[2].get_text()[0:5] == 'Sorry':
        print(str(i) + ' - NO WELL')
    else:
        for index, item in enumerate(fields):
            if item.get_text().strip() == 'Spud Date:':
                if int(fields[index+1].get_text()[-4:]) > 2004:
                    #print(str(i) + ' - DATA GATHERED')
                    spud_d = {'API': i, 'Spud Date': fields[index+1].get_text()}
                    api_list.append(spud_d)

In [5]:
api_list

[{'API': 12346120, 'Spud Date': '2/13/2018'}]

In [10]:
def create_api_list():
    api_list = []
    for i in range(12346120, 12346123):
        url = "http://cogcc.state.co.us/cogis/FacilityDetail.asp?facid=" + str(i) + "&type=WELL"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        fields = soup.find_all('font')
        result = []
        spud_d = {}
        
        if fields[2].get_text()[0:5] == 'Sorry':
            print(str(i) + ' - NO WELL')
        else:
            for index, item in enumerate(fields):
                if item.get_text().strip() == 'Spud Date:':
                    if int(fields[index+1].get_text()[-4:]) > 2004:
                        #print(str(i) + ' - DATA GATHERED')
                        spud_d = {'API': i, 'Spud Date': fields[index+1].get_text()}
                        api_list.append(spud_d)

In [11]:
#import line_profiler
lp = line_profiler.LineProfiler()
lp.add_function(create_api_list)
lp.runctx('create_api_list()', locals=locals(), globals=globals())
lp.print_stats()

Timer unit: 1e-06 s

Total time: 13.5922 s
File: <ipython-input-10-a1dcd3ad9a67>
Function: create_api_list at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_api_list():
     2         1            4      4.0      0.0      api_list = []
     3         4            8      2.0      0.0      for i in range(12346120, 12346123):
     4         3           12      4.0      0.0          url = "http://cogcc.state.co.us/cogis/FacilityDetail.asp?facid=" + str(i) + "&type=WELL"
     5         3     13355075 4451691.7     98.3          page = requests.get(url)
     6         3       229822  76607.3      1.7          soup = BeautifulSoup(page.content, 'html.parser')
     7         3         2598    866.0      0.0          fields = soup.find_all('font')
     8         3            4      1.3      0.0          result = []
     9         3            3      1.0      0.0          spud_d = {}
    10                       

In [12]:
fields

[<font face="Arial" size="2">
 <!--
 <img SRC="images/s_cogcc_head.jpg" width="513" height="51" alt="Colorado Oil & Gas Conservation Commission"><br>
 <img SRC="images/s_head_fill.jpg" width="123" height="22">
  -->
 <p><font color="#000080" face="Arial" size="5"><b>     
  COGIS - WELL Information</b></font></p>
 <!-- BEGIN OUTPUT TO SCREEN -->
 <!-- BEGIN SURFACE INFORMATION -->
 <!-- HANDLE BAD API NUMBER -->
 <table border="0" cellpadding="1" cellspacing="1">
 <tr>
 <td bgcolor="#ffffcc" colspan="4">
 <font size="2">
 <table>
 <tr>
 <td valign="top"><font color="Navy" size="4">Scout Card</font>             </td>
 <!-- -- remove comment tags from the next line and add tags to the wellbore diagram line to disable wellbore diagrams ---->
 <!--&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
 				&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;

In [15]:
def find_2nd(string, substring):
    return string.find(substring, string.find(substring) + 1)

ll_data = []
#range(12300000, 12400000)
#range(12339899, 12339902)

for i in range(12314631, 12314635):
    url = "http://cogcc.state.co.us/cogis/FacilityDetail.asp?facid=" + str(i) + "&type=WELL"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    fields = soup.find_all('font')

    if fields[2].get_text()[0:5] == 'Sorry':
        print(str(i) + ' - NO WELL')
    else:
        ll_dict = {}
        ls = '-0123456789.'
        api = i
        ind = []
        
        for index, item in enumerate(fields):
            if item.get_text().strip()[:3] == 'Lat':
                ind.append(index)
                n = ind[0]
                s = unicodedata.normalize('NFKD', fields[n].get_text().strip())

        for c, l in enumerate(s):
            if l in ls:
                start = c
                slash = find_2nd(s, '/')          
                break
        
        lat = s[start:(slash-1)]
        long = s[(slash+1):]
        
        ll_dict = {'API': api, 'Lat': lat, 'Long': long}
        ll_data.append(ll_dict)
        print(str(i) + ' - DATA GATHERED')

12314631 - DATA GATHERED
12314632 - DATA GATHERED
12314633 - DATA GATHERED
12314634 - DATA GATHERED


In [16]:
ll_data

[{'API': 12314631, 'Lat': '40.46178', 'Long': '-104.755882'},
 {'API': 12314632, 'Lat': '40.46542', 'Long': '-104.760572'},
 {'API': 12314633, 'Lat': '40.21589', 'Long': '-104.76031'},
 {'API': 12314634, 'Lat': '40.12759', 'Long': '-104.814691'}]

In [17]:
cols = ['Well Name/No:', 'Casing:', 'Cement:']

cc_data = []
#range(12300000, 12400000)

for i in range(12332493, 12332496):
    url = "http://cogcc.state.co.us/cogis/FacilityDetail.asp?facid=" + str(i) + "&type=WELL"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    fields = soup.find_all('font')

    m = len(fields)  #in order to not look past more f than are in fields
    d = {}  #dictionary to hold field index and col heading values
    num = []  #list for the field index for each col heading for each API whose value is also in original cols list
    val = []  #list for all the col headings that are also in original cols list
    api = []
    
    for a in range(0, m):
        ''' 
        create dictionary with field indexes as the keys and potential column names as values
        '''
        d[a] = (unicodedata.normalize('NFKD', fields[a].get_text().strip()))

    for key in d:
        '''
        go through dictionary (d) and only keep the keys and values for the fields that are in the original cols list
        '''
        if d[key] in cols:
            num.append(key)
            val.append(d[key])
    
    num.append
    new_d = dict(zip(num, val))  #new dictionary with just field indexes and col names from original cols list
    
    row = {}
    new_d[0] = 'API'  #replace first item in dictionary (unneeded data) with API as key

    for k, v in new_d.items():   
        row[new_d[0]] = i   # add API # as value
    api.append(row)
 
    casing_cols_per = []
    cement_cols_per = []
    casing_vals_per = []
    cement_vals_per = []

    casing_cols_com = []
    cement_cols_com = []
    casing_vals_com = []
    cement_vals_com = []
    
    for index, item in enumerate(fields):
        if item.get_text().strip() == 'Wellbore Completed':
            wc = index 
    
    c = 0
    for k, v in new_d.items():
        if v == 'Casing:' and k < wc:
            c += 1
            casing_cols_per.append(v[:-1] + '_' + str(c) + '_WellPermit')
            casing_vals_per.append(fields[k+1].get_text().strip())
    
    c = 0
    for k, v in new_d.items():
        if v == 'Cement:' and k < wc:
            c += 1
            cement_cols_per.append(v[:-1] + '_' + str(c) + '_WellPermit')
            cement_vals_per.append(fields[k+1].get_text().strip())
                       
    c = 0
    for k, v in new_d.items():
        if v == 'Casing:' and k > wc:
            c += 1
            casing_cols_com.append(v[:-1] + '_' + str(c) + '_WellComplete')
            casing_vals_com.append(fields[k+1].get_text().strip())
    
    c = 0
    for k, v in new_d.items():
        if v == 'Cement:' and k > wc:
            c += 1
            cement_cols_com.append(v[:-1] + '_' + str(c) + '_WellComplete')
            cement_vals_com.append(fields[k+1].get_text().strip())
            
    casing_d_per = dict(zip(casing_cols_per, casing_vals_per))
    cement_d_per = dict(zip(cement_cols_per, cement_vals_per))
    casing_d_com = dict(zip(casing_cols_com, casing_vals_com))
    cement_d_com = dict(zip(cement_cols_com, cement_vals_com))
    
    c_dict = {**api[0], **casing_d_per, **cement_d_per, **casing_d_com, **cement_d_com}
    
    cc_data.append(c_dict)
    
    c_df = pd.DataFrame(cc_data)
    c_df.set_index('API', inplace=True)
    c_df.fillna('', inplace=True)
    #c_df = c_df[['Casing_1_WellPermit', 'Cement_1_WellPermit', 'Casing_2_WellPermit', 'Cement_2_WellPermit',
    #             'Casing_3_WellPermit', 'Cement_3_WellPermit', 'Casing_4_WellPermit', 'Cement_4_WellPermit',
    #             'Casing_1_WellComplete', 'Cement_1_WellComplete', 
    #             'Casing_2_WellComplete', 'Cement_2_WellComplete', 
    #             'Casing_3_WellComplete', 'Cement_3_WellComplete',
    #             'Casing_4_WellComplete', 'Cement_4_WellComplete']]

In [18]:
c_df.T

API,12332493,12332494,12332495
Casing_10_WellPermit,"String Type: SURF , Hole Size: 12.75, Siz...",,
Casing_11_WellPermit,"String Type: 1ST , Hole Size: 8.75, Size...",,
Casing_12_WellPermit,"String Type: 2ND , Hole Size: 6, Size: 4...",,
Casing_13_WellPermit,"String Type: 1ST LINER , Hole Size: 6, Size: 4...",,
Casing_1_WellComplete,"String Type: SURF , Hole Size: 13.5, Size...","String Type: SURF , Hole Size: 12.25, Siz...","String Type: SURF , Hole Size: 12.25, Siz..."
Casing_1_WellPermit,"String Type: SURF , Hole Size: 12.75, Siz...","String Type: SURF , Hole Size: 12.25, Siz...","String Type: SURF , Hole Size: 12.25, Siz..."
Casing_2_WellComplete,"String Type: 1ST , Hole Size: 8.75, Size...","String Type: 1ST , Hole Size: 7.875, Siz...","String Type: 1ST , Hole Size: 7.875, Siz..."
Casing_2_WellPermit,"String Type: OPEN HOLE , Hole Size: 8.75, Size...","String Type: 1ST , Hole Size: 7.875, Siz...","String Type: 1ST , Hole Size: 7.875, Siz..."
Casing_3_WellComplete,"String Type: 1ST LINER , Hole Size: 6, Size: 4...",,
Casing_3_WellPermit,"String Type: SURF , Hole Size: 13.5, Size...",,"String Type: 1ST LINER , Hole Size: , Size: , ..."


In [19]:
cols = ['Well Name/No:', 'Operator:', 'Status Date:', 'FacilityID:', 'LocationID:', 
        'County:', 'Location:', 'Field:', 'Elevation:', 'Spud Date:', 'Spud Date is:',
        'Wellbore Permit', 'Permit #:', 'Expiration Date:', 'Prop Depth/Form:', 'Surface Mineral Owner Same:',
        'Mineral Owner:', 'Surface Owner:', 'Unit:', 'Unit Number:', 'Formation and Spacing:', 
        #'Casing:', 'Cement:', 'Casing:', 'Cement:', 'Casing:', 'Cement:', 'Casing:', 'Cement:',
        'Completion Date:', 'Measured TD:', 'Measured PB depth:', 'True Vertical TD:', 'True Vertical PB depth:', 
        'Log Types:', 'Casing:', 'Cement:', 'Casing:', 'Cement:', 'Casing:', 'Cement:', 'Casing:', 'Cement:',
        '1st Production Date:', 'Choke Size:', 'Status Date:', 'Open Hole Completion:', 'Commingled:', 
        'Production Method:', 'Formation Name:', 'Status:', 'Formation Treatment', 'Treatment Type:',
        'Treatment Date:', 'Treatment End Date:', 'Treatment Summary:', 'Total fluid used in treatment (bbls):',
        'Max pressure during treatment (psi):', 'Total gas used in treatment (mcf):', 'Fluid density (lbs/gal):',
        'Type of gas:', 'Number of staged intervals:', 'Total acid used in treatment (bbls):', 
        'Min frac gradient (psi/ft):', 'Recycled water used in treatment (bbls):', 
        'Total flowback volume recovered (bbls):', 'Produced water used in treatment (bbls):', 
        'Disposition method for flowback:', 'Total proppant used (lbs):', 'Green completions techniques utilized:',
        'Reason green techniques not utilized:', 'Tubing Size:', 'Tubing Setting Depth:', 'Tubing Packer Depth:', 
        'Tubing Multiple Packer:', 'Open Hole Top:', 'Open Hole Bottom:', 'Test Date:', 'Test Method:', 
        'Hours Tested:', 'Gas Type:', 'Gas Disposal:', 'BBLS_H2O', 'BBLS_OIL', 'BTU_GAS', 'CALC_BBLS_H2O', 
        'CALC_BBLS_OIL', 'CALC_GOR', 'CALC_MCF_GAS', 'CASING_PRESS', 'GRAVITY_OIL', 'MCF_GAS', 'TUBING_PRESS',       
        'Interval Bottom:', 'Interval Top:', '# of Holes:', 'Hole Size:']

def find_2nd(string, substring):
    return string.find(substring, string.find(substring) + 1)

data = []

for i in range(12332493, 12332496):
    url = "http://cogcc.state.co.us/cogis/FacilityDetail.asp?facid=" + str(i) + "&type=WELL"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    fields = soup.find_all('font')
    
    if fields[2].get_text()[0:5] == 'Sorry':
        print(str(i) + ' - NO WELL')
    else:
        m = len(fields)  #in order to not look past more f than are in fields
        d = {}  #dictionary to hold field index and col heading values
        num = []  #list for the field index for each col heading for each API whose value is also in original cols list
        val = []  #list for all the col headings that are also in original cols list
        ll_dict = {}
        ls = '-0123456789.'
        api = i
        ind = []
        
        for index, item in enumerate(fields):
            '''
            go through web data and find the 2nd mention of 'Lat/Long' and save that string 
            '''
            if item.get_text().strip()[:10] == 'As Drilled':
                ind.append(index)
                n = ind[0]+1
                s = unicodedata.normalize('NFKD', fields[n].get_text().strip())

        for c, l in enumerate(s):
            '''
            go through 2nd mention of 'Lat/Long' string and find where the lat number starts (start) and where
            the long number starts (slash)
            '''            
            if l in ls:
                start = c
                slash = find_2nd(s, '/')          
                break
        
        lat = s[start:(slash-1)]
        long = s[(slash+1):]

        for a in range(0, m):
            ''' 
            create dictionary with field indexes as the keys and potential column names as values
            '''
            d[a] = (unicodedata.normalize('NFKD', fields[a].get_text().strip()))

        for k in d:
            '''
            go through dictionary (d) and only keep the keys and values for the fields that are in the original cols list
            '''
            if d[k] in cols:
                num.append(k)
                val.append(d[k])

        num.append
        new_d = dict(zip(num, val))  #new dictionary with just field indexes and col names from original cols list

        row = {}
        new_d[0] = 'API'  #make first item in dictionary API code

        for k, v in new_d.items():   
            row[new_d[k]] = unicodedata.normalize('NFKD', fields[k+1].get_text().strip())
            row[new_d[0]] = i   # add API # as value
            row['Lat'] = lat
            row['Long'] = long
        data.append(row)
        print(str(i) + ' - DATA GATHERED')

        data_df = pd.DataFrame(data)
        data_df.set_index('API', inplace=True)
        data_df.fillna('', inplace=True)

12332493 - DATA GATHERED
12332494 - DATA GATHERED
12332495 - DATA GATHERED


In [20]:
data_df.head().T

API,12332493,12332494,12332495
# of Holes:,,28,112
1st Production Date:,3/12/2011,N/A,9/1/2011
BBLS_H2O,138,13,
BBLS_OIL,137,23,40
BTU_GAS,1303,1303,1267
CALC_BBLS_H2O,138,13,
CALC_BBLS_OIL,137,23,40
CALC_GOR,2380,4174,5000
CALC_MCF_GAS,326,96,200
CASING_PRESS,505,787,1650


In [212]:
well_df = pd.merge(data_df, c_df, left_index=True, right_index=True)
well_df = pd.merge(well_df, ll_df, left_index=True, right_index=True)

In [213]:
well_df

,# of Holes:,1st Production Date:,BBLS_H2O,BBLS_OIL,BTU_GAS,CALC_BBLS_H2O,CALC_BBLS_OIL,CALC_GOR,CALC_MCF_GAS,CASING_PRESS,...,Cement_1_WellComplete,Cement_1_WellPermit,Cement_2_WellComplete,Cement_2_WellPermit,Cement_3_WellComplete,Cement_3_WellPermit,Cement_4_WellComplete,Cement_4_WellPermit,Lat,Long
API,,,,,,,,,,,,,,,,,,,,,
12339899,0,6/1/2015,458,906,1264,458,906,197,179,836,...,"Sacks: 64, Top: 0, Bottom: 116, Method Grade: ...","Sacks: 6, Top: 0, Bottom: 100, Method Grade:","Sacks: 463, Top: 0, Bottom: 1213, Method Grade...","Sacks: 520, Top: 0, Bottom: 1150, Method Grade:","Sacks: 438, Top: 1240, Bottom: 5859, Method Gr...","Sacks: 400, Top: 0, Bottom: 5820, Method Grade:","Sacks: , Top: , Bottom: , Method Grade:","Sacks: , Top: , Bottom: , Method Grade:",40.77445,-103.82038


In [48]:
data_df.to_csv('well_data.csv')